In [ ]:
!pip install pennylane

In [ ]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt

In [88]:
#Definimos el tamaño del problema
numeroUsuarios = 4
numeroVoluntarios = 5
rango = max(numeroUsuarios, numeroVoluntarios)
nqubits = numeroVoluntarios * numeroUsuarios
distancia = [[0 for i in range(rango)] for j in range(rango)]
peticionUsuarios = np.random.randint(0, 2, numeroUsuarios)
peticionVoluntarios = np.random.randint(0, 2, numeroVoluntarios)
for i in range(numeroVoluntarios):
  for j in range(numeroUsuarios):
    distancia[i][j] = np.random.randint(5, 500)

In [112]:
def toLineal(i, j):
  return i*numeroUsuarios + j

In [113]:
def fromLineal(n):
  return n//numeroUsuarios, n%numeroUsuarios

In [89]:
print(peticionUsuarios)
print(peticionVoluntarios)
print(distancia)

[1 1 1 0]
[1 1 0 0 1]
[[238, 232, 346, 73, 0], [257, 256, 236, 292, 0], [483, 306, 269, 461, 0], [217, 366, 307, 390, 0], [291, 454, 313, 253, 0]]


In [122]:
def matrizObjetivo():
  objetivo = [[0 for i in range(nqubits)] for j in range(nqubits)]
  for i in range(nqubits):
    for j in range(nqubits):
      v1,u1 = fromLineal(i)
      v2,u2 = fromLineal(j)
      if i == j:
        objetivo[i][j] = (distancia[v1][u1] * peticionVoluntarios[v1] * peticionUsuarios[u1] - (2 * peticionVoluntarios[v1]) - (2 *  peticionUsuarios[u1]) - 2)/2
      else:
        objetivo[i][j] = 0.5 * peticionVoluntarios[v1] * peticionUsuarios[u1]* peticionVoluntarios[v2] * peticionUsuarios[u2]
  return objetivo

In [15]:
dev = qml.device("default.qubit", wires = range(nqubits))

In [10]:
def U_C(gamma, h, n):
  for i in range(n):
    for j in range(n):
        if (j != i):
          qml.CNOT(wires = [i, j])
          qml.RZ(-2 * (-h[i][j] * h[i][j]) * gamma, wires = j)
          qml.CNOT(wires = [i, j])

In [11]:
def U_B(beta, h, n):
  for k in range(n):
    qml.RX(-2 * beta * h[k][k], wires = k)

In [17]:
@qml.qnode(dev)
def circuito(gamma, beta, h, n, capas):
  for k in range(n):
    qml.Hadamard(wires = k)
  for k in range(capas):
    U_C(gamma, h, n)
    U_B(beta, h, n)
  return qml.probs(wires = range(n))

In [13]:
gamma = 0.15
beta = 0.15
capas = 3

In [123]:
resultados = circuito(gamma, beta, matrizObjetivo(), nqubits, capas)

In [27]:
def getMaxKey(valores):
  maxValue = max(valores)
  for i in range(2**nqubits):
    if valores[i] == maxValue:
      return i

In [124]:
ganador=getMaxKey(resultados)
print(ganador)

434178


In [35]:
def decimalToBinary(n):
    return bin(n).replace("0b", "")

In [125]:
estadoGanador = decimalToBinary(ganador)

In [127]:
for i in range(len(str(estadoGanador))):
  if (estadoGanador[i] == '1'):
    u, v = fromLineal(i)
    print(f"El voluntario %1d debe acudir a visitar al usuario %1d" % (u+1, v+1))

El voluntario 1 debe acudir a visitar al usuario 1
El voluntario 2 debe acudir a visitar al usuario 2
El voluntario 5 debe acudir a visitar al usuario 3
